In [183]:
import numpy as np
from numpy.random import choice

In [184]:
n, m = 4, 4
a = np.random.randn(n) ** 2
b = np.random.randn(m) ** 2
a /= np.sum(a)
b /= np.sum(b)

In [185]:
cost = np.random.randn(n, m) ** 2

In [204]:
def perturb(p, k=10):
    p = p.copy()
    n, m = p.shape

    for _ in range(k):
        i1, i2 = choice(range(n), size=2, replace=False)
        j1, j2 = choice(range(n), size=2, replace=False)

        if cost[i1, j1] > cost[i1, j2]:
            j1, j2 = j2, j1

        swap = min(1-p[i1, j1], 1-p[i2, j2], p[i1, j2], p[i2, j1]) * np.random.rand()

        p[i1, j1] += swap
        p[i1, j2] -= swap
        p[i2, j1] -= swap
        p[i2, j2] += swap

    return p

In [187]:
def marry(p, q):
    lmbda = np.random.rand()
    return p * lmbda + (1-lmbda) * q

In [188]:
def eval_cost(p):
    return (p * cost).sum()

In [209]:
fam_size = 100
n_ch = 2

In [190]:
from tqdm import tqdm_notebook as tqdm


In [191]:
from IPython.display import clear_output

In [219]:
initial = np.multiply.outer(a,b)
for _ in tqdm(range(10000)):
    family = np.array([initial.copy() for _ in range(fam_size)])

    new_family = np.zeros((fam_size * n_ch, n,m))

    for i, x in enumerate(family):
        for j, y in enumerate(family[np.random.randint(low=0, high=fam_size, size=n_ch),:,:]):
            new_family[i * n_ch + j, :, :] = perturb(marry(x,y), k=10)
        
    fitness = list(map(lambda p : eval_cost(p), new_family))
    survivors = np.argpartition(fitness, fam_size)[:fam_size]
    print('Generation', _, np.min(fitness))
    print(np.mean(fitness))
    clear_output(wait=True)
    family = np.array(new_family)[survivors]


KeyboardInterrupt: 

In [212]:
import ot

In [198]:
(ot.emd(a, b, cost) * cost).sum()

0.014192307381813157

In [180]:
eval_cost(initial)

0.175803649808498

In [197]:
(ot.sinkhorn(a,b,cost,reg=.001) * cost).sum()

0.014190013935294673